In [1]:
import pandas as pd
from tqdm import tqdm
import itertools
import numpy as np

In [2]:
data = pd.read_pickle('../processed/data_with_features.pkl')

In [3]:
data.shape

(29622473, 131)

In [4]:
def print_groupby(iterable, keyfunc=None):
    for k, g in itertools.groupby(iterable, keyfunc):
        print("key: '{}'--> group: {}".format(k, list(g)))

In [5]:
def groupby_mark_for_ommit(iterable, keyfunc=None, threshold = 3, size = 0.4):
    array_to_drop = []
    for k, g in itertools.groupby(iterable, keyfunc):
        g_len = len(list(g))
        if( g_len>=threshold):
            mask=np.array( [False]*g_len)
            inds=np.random.choice(np.arange(g_len),size=  int(g_len * size)  )
            mask[inds]=True
            array_to_drop.append(mask)
            #print("key: '{}'--> group: {}".format(k, list(g)))
        else:
            array_to_drop.append([False]*g_len)
    return array_to_drop

In [6]:
unique_SKU_list = list(data.SKU.unique())
for SKU_ in tqdm(unique_SKU_list):
    tmp_subset = data[data['SKU'] == SKU_]
    tmp_result = groupby_mark_for_ommit(tmp_subset.sales.values, None, 3, 0.9)
    flat_list = [item for sublist in tmp_result for item in sublist]
    tmp_subset['to_drop'] = flat_list
    # assign to main DF
    data.loc[data['SKU'] == SKU_,'to_drop'] = tmp_subset['to_drop']

  0%|                                                  | 0/1961 [00:00<?, ?it/s]/tmp/ipykernel_29143/4218711918.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_subset['to_drop'] = flat_list
100%|███████████████████████████████████████| 1961/1961 [24:04<00:00,  1.36it/s]


In [7]:
data.to_pickle('../processed/data_with_features2.pkl')

In [8]:
data['to_drop'].unique()

array([False, True], dtype=object)